# Installing packages section

In [ ]:
!pip install --upgrade pip
!pip install "snowflake-connector-python[pandas]" "snowflake-snowpark-python[pandas]" snowflake-snowpark-python==1.9.0 numpy pandas matplotlib scikit-learn xgboost seaborn python-dateutil tqdm holidays faker
!pip install --upgrade --q snowflake-snowpark-python==1.9.0
!pip uninstall urllib3 -y
!pip install urllib3==1.26.15
!pip install fosforml==1.1.6

# Importing packages

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
import joblib
#from fosforio import snowflake
from fosforml import *
from fosforml.constants import MLModelFlavours
#from fosforio import get_dataframe
from matplotlib import pyplot as plt
import pandas as pd
pd.set_option('display.max_columns', 500)
import seaborn as sns
from sklearn.metrics import mean_absolute_percentage_error
import numpy as np
import warnings; warnings.simplefilter('ignore')
from joblib import dump, load
import requests
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression
import configparser
from dateutil.relativedelta import relativedelta
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score
%matplotlib inline

# Fetching the datasets

In [3]:
from fosforml.model_manager.snowflakesession import get_session
my_session = get_session()

In [4]:
table_name = 'DF_ALL'
df_all = my_session.sql("select * from {}".format(table_name))
df_all = df_all.to_pandas()
df_all

,AD_DATE,TOTAL_IMPRESSIONS,TOTAL_REVENUE,VIEWABLE_IMPRESSIONS,MEASURABLE_IMPRESSIONS,AD_TYPE,AD_FORMAT,AD_MEDIA_TYPE,DEVICE_TYPE,CITY,POPULATION,CITY_LAT,CITY_LON,LINE_ITEM_GROUP,LINE_ITEM_TYPE,MONETIZATION_CHANNEL,OS_TYPE,SITE_ID,ADVERTISER_ID
0,16-06-2024,122,0.3712,56,122,interactive,Interstitial,gif,laptop,New York,8258035,40.66,-73.94,non_guaranteed,ad_exchange,in_app_advertising,ios,343,8
1,16-06-2024,170,0.5224,56,170,interactive,in_content,shorts,laptop,New York,8258035,40.66,-73.94,non_guaranteed,ad_exchange,in_app_advertising,ios,351,8
2,16-06-2024,172,0.6732,56,170,interactive,in_content,survey,tablet,New York,8258035,40.66,-73.94,non_guaranteed,ad_exchange,in_app_advertising,android,351,8
3,16-06-2024,120,0.0000,58,120,interactive,banner_customizable,html5,phone,Los Angeles,3820914,34.02,-118.41,non_guaranteed,adsense,sponsorship,mac,347,2639
4,16-06-2024,50,0.0000,58,50,non_interactive,native_rich,content,laptop,New York,8258035,40.66,-73.94,non_guaranteed,ad_exchange,in-app_purchases,ios,346,79
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
567286,15-07-2024,28,0.1822,6,22,interactive,rewarded interstitial_customizable,html5,laptop,El Paso,678958,31.85,-106.43,guaranteed,standard,in-app_purchases,ios,346,79
567287,15-07-2024,156,0.2130,6,156,interactive,retargetted,gif,laptop,New York,8258035,40.66,-73.94,non_guaranteed,ad_exchange,in_app_advertising,windows,349,8
567288,15-07-2024,108,0.2160,6,108,interactive,in_feed,survey,laptop,New York,8258035,40.66,-73.94,non_guaranteed,ad_exchange,sponsorship,windows,350,2634
567289,15-07-2024,122,0.2174,6,122,interactive,banner_customizable,gif,laptop,New York,8258035,40.66,-73.94,non_guaranteed,ad_exchange,in_app_advertising,ios,343,90


In [6]:
table_name = 'DF_OPT'
df_opt = my_session.sql("select * from {}".format(table_name))
df_opt = df_opt.to_pandas()
df_opt

,AD_DATE,DEVICE_TYPE,CITY,POPULATION,CITY_LAT,CITY_LON,MONETIZATION_CHANNEL,OS_TYPE,SITE_ID,ADVERTISER_ID,TOTAL_REVENUE,TOTAL_IMPRESSIONS,VIEWABLE_IMPRESSIONS,MEASURABLE_IMPRESSIONS,AD_TYPE,AD_FORMAT,AD_MEDIA_TYPE,LINE_ITEM_GROUP,LINE_ITEM_TYPE
0,16-06-2024,phone,Los Angeles,3820914,34.02,-118.41,sponsorship,mac,347,2639,0.0003,0,0,0,interactive,banner_customizable,html5,non_guaranteed,ad_exchange
1,16-06-2024,phone,Los Angeles,3820914,34.02,-118.41,sponsorship,mac,347,2637,0.0003,0,0,0,interactive,Interstitial,shorts,non_guaranteed,ad_exchange
2,16-06-2024,tablet,Chicago,2664452,41.84,-87.68,sponsorship,android,351,2645,0.0016,0,0,0,interactive,in_content,rich_media,non_guaranteed,ad_exchange
3,16-06-2024,phone,New York,8258035,40.66,-73.94,sponsorship,mac,351,2646,0.0009,0,0,0,interactive,in_content,rich_media,non_guaranteed,ad_exchange
4,16-06-2024,tablet,Chicago,2664452,41.84,-87.68,freemium,android,345,2636,0.0005,0,0,0,interactive,rewarded,audio,non_guaranteed,ad_exchange
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
567286,15-07-2024,laptop,El Paso,678958,31.85,-106.43,in-app_purchases,ios,346,79,0.1822,28,6,22,interactive,rewarded interstitial_customizable,html5,guaranteed,standard
567287,15-07-2024,laptop,New York,8258035,40.66,-73.94,in_app_advertising,windows,349,8,0.2130,156,6,156,interactive,retargetted,gif,non_guaranteed,ad_exchange
567288,15-07-2024,laptop,New York,8258035,40.66,-73.94,sponsorship,windows,350,2634,0.2160,108,6,108,interactive,in_feed,survey,non_guaranteed,ad_exchange
567289,15-07-2024,laptop,New York,8258035,40.66,-73.94,in_app_advertising,ios,343,90,0.2174,122,6,122,interactive,banner_customizable,gif,non_guaranteed,ad_exchange


In [ ]:
# Old method of bringing in data from SF
#from fosforio import snowflake
#from fosforio import get_dataframe

#snowflake.get_connection(connection_name="ME_AD_SALES_CXN")

In [ ]:
#df_all = get_dataframe("DF_ALL")
#df_opt = get_dataframe("DF_OPT")

In [ ]:
#df_all.info()

In [ ]:
#df_opt.info()

In [7]:
df_opt.columns = df_opt.columns.str.lower()
df_all.columns = df_all.columns.str.lower()

# Exploratory data analysis

In [ ]:
cat_col=['date','geo_id','order_id','site_id', 'ad_type_id','device_type', 'advertiser_id',
         'line_item_type', 'os_type','monetization_channel']
scat_col = ['site_id', 'ad_type','device_type', 'advertiser_id',
         'line_item_type', 'os_type','monetization_channel']
num_col=list(df_all.select_dtypes(np.number).columns)

In [ ]:
null_counts = df_all.isnull().sum()

null_counts = null_counts[null_counts > 0]

if not null_counts.empty:
    plt.figure(figsize=(12, 8))
    null_counts.plot(kind='bar', color='skyblue')
    plt.title('Count of Null Values in Each Column')
    plt.xlabel('Columns')
    plt.ylabel('Number of Null Values')
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.show()
else:
    print("There are no null values in the DataFrame.")

# bar plot

In [ ]:
fig, ax = plt.subplots(4, 2, figsize=(14, 12))

axes_ = [axes_row for axes in ax for axes_row in axes]

for i, col in enumerate(scat_col):
    if col in df_all.columns:
        sns.countplot(data=df_opt, x=col, ax=axes_[i])
        if col == 'advertiser_id':
            axes_[i].set_xticklabels(axes_[i].get_xticklabels(), rotation=90)
    else:
        axes_[i].text(0.5, 0.5, f'Column {col} not found', ha='center', va='center')
        axes_[i].set_axis_off()

plt.tight_layout()

plt.show()

# Scatter plot using total_revenue

In [ ]:
for i in num_col:
    if i!='total_revenue':
        sns.scatterplot(data=df_opt,x=i,y='total_revenue')
        plt.show()

In [ ]:
for col in (scat_col):
    title='Relationship of '+col+' with total_revenue'
    plt.figure(figsize=(6,4))
    sns.barplot(y=df_all['total_revenue'],x=df_all[col])
    if col=='advertiser_id':
        plt.xticks(rotation=90)
    plt.title(title)
    plt.show()

In [ ]:
for col in (scat_col):
    title='Relationship of '+ col +' with total_impressions'
    plt.figure(figsize=(6,4))
    sns.barplot(y=df_all['total_impressions'],x=df_all[col],)
    if col=='advertiser_id':
        plt.xticks(rotation=90)
    plt.title(title)
    plt.show()

In [ ]:
for i in df_all.select_dtypes(object).columns:
    pd.crosstab(df_all['monetization_channel'],df_all[i]).plot(kind='bar')
    plt.show()